# Link Prediction

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.0+cu113
     |████████████████████████████████| 7.9 MB 4.1 MB/s 
     |████████████████████████████████| 3.5 MB 3.7 MB/s 
     |████████████████████████████████| 2.4 MB 4.2 MB/s 


In [3]:
import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

## データセットCora

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(
        num_val=0.05,  # for validation
        num_test=0.1,  # for test
        is_undirected=True,
        add_negative_train_samples=False), # no negative sample added
])

dataset = Planetoid('data/Planetoid',  name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]

Processing...
Done!


In [5]:
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[4488], edge_label_index=[2, 4488])

In [ ]:
train_data['edge_label']

tensor([1., 1., 1.,  ..., 1., 1., 1.])

問題

- 特定の論文間に参照・非参照の関係（=1）が与えられていて
- データの一部に対し、接続関係および接続関係にある頂点内特徴ベクトルを学習データとして学習し、
- 学習結果を使い、残りのデータに対し、リンク推定を行う、結果を評価する。(validation, test)


In [9]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [11]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)
final_edge_index

Epoch: 001, Loss: 0.6930, Val: 0.7117, Test: 0.7099
Epoch: 002, Loss: 0.6809, Val: 0.7139, Test: 0.6999
Epoch: 003, Loss: 0.7159, Val: 0.7233, Test: 0.7037
Epoch: 004, Loss: 0.6771, Val: 0.7405, Test: 0.7171
Epoch: 005, Loss: 0.6854, Val: 0.7610, Test: 0.7433
Epoch: 006, Loss: 0.6894, Val: 0.7659, Test: 0.7688
Epoch: 007, Loss: 0.6906, Val: 0.7152, Test: 0.7342
Epoch: 008, Loss: 0.6905, Val: 0.6854, Test: 0.7081
Epoch: 009, Loss: 0.6894, Val: 0.6839, Test: 0.7020
Epoch: 010, Loss: 0.6869, Val: 0.6932, Test: 0.7015
Epoch: 011, Loss: 0.6830, Val: 0.7012, Test: 0.7010
Epoch: 012, Loss: 0.6793, Val: 0.7060, Test: 0.6997
Epoch: 013, Loss: 0.6789, Val: 0.7103, Test: 0.6995
Epoch: 014, Loss: 0.6783, Val: 0.7156, Test: 0.6998
Epoch: 015, Loss: 0.6744, Val: 0.7215, Test: 0.7009
Epoch: 016, Loss: 0.6680, Val: 0.7258, Test: 0.7016
Epoch: 017, Loss: 0.6656, Val: 0.7273, Test: 0.7001
Epoch: 018, Loss: 0.6622, Val: 0.7269, Test: 0.6976
Epoch: 019, Loss: 0.6549, Val: 0.7232, Test: 0.6953
Epoch: 020, 

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    1,    2,  ..., 2704, 2706, 2707]])

In [13]:
(test_data.edge_index).shape

torch.Size([2, 9502])

In [14]:
final_edge_index.shape

torch.Size([2, 3343402])